# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv('cities_data.csv')
df.dropna(inplace=True)
df

,City,lat,long,temp,humidity,cloudiness,windspeed
0,seydi,39.4816,62.9137,69.48,19.0,0.0,16.91
1,barrow,71.2906,-156.7887,17.60,79.0,90.0,12.66
2,longyearbyen,78.2186,15.6401,28.40,74.0,75.0,16.11
3,sitka,57.0531,-135.3300,44.60,87.0,90.0,12.66
4,fairbanks,64.8378,-147.7164,61.50,23.0,40.0,3.60
...,...,...,...,...,...,...,...
607,jining,35.4050,116.5814,67.28,85.0,100.0,8.75
608,waingapu,-9.6567,120.2641,81.10,75.0,94.0,15.01
610,vicuna,-30.0319,-70.7081,63.00,37.0,5.0,1.01
611,sinop,41.7711,34.8709,61.68,57.0,6.0,6.87


In [3]:
map_df=df[["lat", "long", "humidity"]]
map_df


,lat,long,humidity
0,39.4816,62.9137,19.0
1,71.2906,-156.7887,79.0
2,78.2186,15.6401,74.0
3,57.0531,-135.3300,87.0
4,64.8378,-147.7164,23.0
...,...,...,...
607,35.4050,116.5814,85.0
608,-9.6567,120.2641,75.0
610,-30.0319,-70.7081,37.0
611,41.7711,34.8709,57.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = map_df[['lat', 'long']]
weights = map_df['humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
df
hotel_df=df[(df['temp'] < 80) & (df['temp'] > 70)]
hotel_df=hotel_df[(hotel_df['windspeed'] < 10) & (hotel_df['cloudiness'] == 0)]
hotel_df


,City,lat,long,temp,humidity,cloudiness,windspeed
328,ranong,9.9658,98.6348,77.00,94.0,0.0,1.14
342,palmas,-10.2128,-48.3603,78.80,73.0,0.0,4.61
365,dakar,14.6937,-17.4441,71.60,88.0,0.0,6.91
472,khuzdar,27.7384,66.6434,71.53,15.0,0.0,5.28
531,sharjah,25.3573,55.4033,79.68,50.0,0.0,4.61
594,nushki,29.5542,66.0215,72.19,15.0,0.0,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df["Hotel Name"]=""
hotel_df["Country Name"]=""
hotel_df

,City,lat,long,temp,humidity,cloudiness,windspeed,Hotel Name,Country Name
328,ranong,9.9658,98.6348,77.00,94.0,0.0,1.14,,
342,palmas,-10.2128,-48.3603,78.80,73.0,0.0,4.61,,
365,dakar,14.6937,-17.4441,71.60,88.0,0.0,6.91,,
472,khuzdar,27.7384,66.6434,71.53,15.0,0.0,5.28,,
531,sharjah,25.3573,55.4033,79.68,50.0,0.0,4.61,,
594,nushki,29.5542,66.0215,72.19,15.0,0.0,1.99,,


In [7]:
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

In [15]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = row["lat"]
    lng = row["long"]
    
    params["location"] = f"{lat},{lng}"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()

    hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

hotel_df.head()

,City,lat,long,temp,humidity,cloudiness,windspeed,Hotel Name,Country Name
328,ranong,9.9658,98.6348,77.00,94.0,0.0,1.14,Bang Rin,
342,palmas,-10.2128,-48.3603,78.80,73.0,0.0,4.61,Palmas,
365,dakar,14.6937,-17.4441,71.60,88.0,0.0,6.91,Dakar,
472,khuzdar,27.7384,66.6434,71.53,15.0,0.0,5.28,Khuzdar,
531,sharjah,25.3573,55.4033,79.68,50.0,0.0,4.61,Sharjah,


In [44]:
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
    lat = row["lat"]
    lng = row["long"]
    
    new_params = {
        "location": f"{lat},{lng}",
        "key": g_key
    }
country_abbrev=requests.get(base_url, params=new_params)
print(country_abbrev)



<Response [400]>


In [ ]:
   
    country_short = requests.get(base_url, params=params)
    country_short = country_short.json()
    #print(json.dumps(country_short, indent=4, sort_keys=True))
    hotel_df.loc[index, "Country"] = country_short['results']['address_components'][3]['short_name']
    #print(results['address_components'][3]['short_name'])

hotel_df.head()

#https://maps.googleapis.com/maps/api/geocode/json?latlng=40.714224,-73.961452&key=YOUR_API_KEY

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
